In [1]:
from GUI import GUI
from layout.obj.Compo import Compo
import layout.lib.pairing as pairing
import layout.lib.repetition_recognition as rep
import layout.lib.draw as draw
from sklearn.cluster import DBSCAN

import pandas as pd
import math
import numpy as np

In [ ]:
gui = GUI(img_file='data/input/4.jpg')

In [2]:
gui = GUI(img_file='E:\\Mulong\\Datasets\\gui\\rico\\combined\\all\\22833.jpg', output_dir='E:\\Temp\\data\\rico-layout\\result')

In [12]:
gui.element_detection()
gui.visualize_element_detection()

[Text Detection Completed in 1.575 s] Input: E:\Mulong\Datasets\gui\rico\combined\all\22833.jpg Output: E:\Temp\data\rico-layout\result\ocr\22833.json
[Compo Detection Completed in 0.684 s] Input: E:\Mulong\Datasets\gui\rico\combined\all\22833.jpg Output: E:\Temp\data\rico-layout\result\ip\22833.json
[Merge Completed] Input: E:\Mulong\Datasets\gui\rico\combined\all\22833.jpg Output: E:\Temp\data\rico-layout\result\uied\22833.jpg


In [3]:
gui.layout_recognition()
gui.visualize_layout_recognition()

[Layout Recognition Completed in 3.984 s] Input: E:\Mulong\Datasets\gui\rico\combined\all\22833.jpg Output: E:\Temp\data\rico-layout\result\layout\22833.json
Thu Aug  5 10:05:59 2021 




In [4]:
gui.compos_df.select_by_class(['Text'], no_parent=True, replace=True)

In [6]:
df = gui.compos_df.compos_dataframe

In [9]:
df['group_nontext'] = -1

In [15]:
df['id']

Series([], Name: id, dtype: object)

In [6]:
gui.cvt_compos_json_to_dataframe()

In [7]:
gui.compos_df.repetitive_group_recognition()

In [5]:
gui.compos_df.pair_groups()

In [7]:
gui.compos_df.repetitive_block_recognition()

In [5]:
b = gui.visualize_compos_df('group')

In [8]:
# gui.compos_df.pair_groups()

In [12]:
# gui.visualize_compos_df('group_pair')

In [2]:
gui = GUI(img_file='E:\\Mulong\\Datasets\\gui\\rico\\combined\\all\\22833.jpg',
          compos_json_file='E:\\Temp\\data\\rico-layout\\result\\uied\\22833.json',
          output_dir='E:\\Temp\\data\\rico-layout\\result')

In [12]:
gui.cvt_compos_json_to_dataframe()

In [13]:
gui.compos_df.compos_dataframe

,id,class,column_min,column_max,row_min,row_max,height,width,area,center,center_column,center_row,text_content,children,parent
0,0,Compo,24,64,35,72,37,40,1480,"(44.0, 53.5)",44.0,53.5,None,"(13,)",None
1,1,Compo,119,150,34,73,39,31,1209,"(134.5, 53.5)",134.5,53.5,None,None,None
2,2,Compo,208,239,35,72,37,31,1147,"(223.5, 53.5)",223.5,53.5,None,None,None
3,3,Compo,297,328,40,73,33,31,1023,"(312.5, 56.5)",312.5,56.5,None,None,None
4,4,Compo,393,406,41,66,25,13,325,"(399.5, 53.5)",399.5,53.5,None,None,None
5,5,Compo,7,442,86,162,76,435,33060,"(224.5, 124.0)",224.5,124.0,None,"(14, 15, 16, 17)",None
6,6,Compo,35,73,701,740,39,38,1482,"(54.0, 720.5)",54.0,720.5,None,"(18,)",None
7,7,Compo,148,186,701,740,39,38,1482,"(167.0, 720.5)",167.0,720.5,None,None,None
8,8,Compo,260,299,705,733,28,39,1092,"(279.5, 719.0)",279.5,719.0,None,None,None
9,9,Compo,373,411,702,740,38,38,1444,"(392.0, 721.0)",392.0,721.0,None,None,None


In [4]:
gui.compos_df.pair_groups()

In [6]:
compos = gui.compos_df
df = compos.compos_dataframe
all_groups = compos.split_groups('group')

In [8]:
g_nt0 = df[df['group'] == 'nt-0']
g_t10 = df[df['group'] == 't-10']

In [9]:
pairing.match_two_groups_by_distance(g_nt0, g_t10)

nt-0 t-10
[289.3579098625092, 333.28966380612525, 309.45758998609165, 308.4234102658227, 339.7484363466593, 339.418620585259, 315.4314505562183, None, None]


TypeError: unorderable types: NoneType() > float()

In [10]:
len(g_nt0)

9

In [11]:
len(g_t10)

10

In [8]:
# gui.layout_recognition(check_valid_group=False)
# gui.visualize_layout_recognition()